In [1]:
import pandas as pd 
train_data  = pd.read_csv('train_data_10_2_4.csv')
test_data = pd.read_csv('test_data_10_2_4.csv')

In [2]:
##添加文章长度
# 
train_data['title_len'] = train_data['title'].map( lambda x : len(x) )

KeyError: 'title'

In [2]:
train_data = train_data.drop(['Unnamed: 0'],axis=1)
test_data = test_data.drop(['Unnamed: 0'],axis=1)

In [3]:
train_X_data = train_data.drop(['label'],axis=1)
train_Y_data = train_data['label']

test_X_data = test_data.drop(['label'],axis=1)
vali_data = test_data['label']

In [4]:
#用xgb跑一下
from pylab import mpl
from matplotlib import pyplot as plt
mpl.rcParams['font.sans-serif'] = ['SimHei'] 
def plot_feature_importance(df,title):
        """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
        df = df.sort_values('importance',ascending = False,).reset_index()
        df['importance_normalized'] = df['importance']/df['importance'].sum()
        
        plt.figure(figsize=(10,6))
        ax = plt.subplot()
        
        #取前15个feature
        ax.barh(list(reversed(list(df.index[:]))), df['importance_normalized'], align = 'center', edgecolor = 'k')
        
        ax.set_yticks(list(reversed(list(df.index[:]))))
        ax.set_yticklabels(df['feature'])
        
        plt.xlabel('Normalized Importance')
        plt.title(title)
        
        plt.show()
        
        return df
import xgboost as xgb
xgb =  xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=9, min_child_weight=5, missing=None, n_estimators=10,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=0,
       subsample=0.8)
model = xgb.fit(train_X_data,train_Y_data,eval_metric='auc')
feature_importance_values = model.feature_importances_
features = list(train_X_data.columns)

xgb_feature_importances = pd.DataFrame({'feature':features,'importance':feature_importance_values})

plot_feature_importance(xgb_feature_importances,'xgb_feature_importances')

<Figure size 1000x600 with 1 Axes>

,index,feature,importance,importance_normalized
0,30,prefix_title_ctr,0.202401,0.202401
1,2,title_pred_0,0.157386,0.157386
2,0,prefix_pred_0,0.142047,0.142047
3,28,prefix_tag_ctr,0.119373,0.119373
4,25,title_ctr,0.099033,0.099033
5,29,title_tag_ctr,0.076359,0.076359
6,26,prefix_ctr,0.056686,0.056686
7,1,prefix_pred_1,0.056352,0.056352
8,3,title_pred_1,0.053018,0.053018
9,31,smi_pre_ti,0.011337,0.011337


In [5]:
from sklearn.metrics import classification_report
xgb_predict_label = model.predict(test_X_data)
print('xgb_predict') 
print(classification_report(vali_data,xgb_predict_label,digits=8))

xgb_predict
             precision    recall  f1-score   support

          0  0.80692925 0.88307016 0.84328449     31412
          1  0.76486781 0.64288405 0.69859089     18585

avg / total  0.79129408 0.79378763 0.78949865     49997



d:\Users\tkhoon\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
